In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import os, cv2, mmcv, torch, cvut

from mmdet.datasets import build_dataloader, build_dataset
from operator import itemgetter 
import tqdm


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
!ln -s /home/member/Workspace/tank/TS_release/dataset/datav2 data
# Build dataset
config_file = "../configs/street/atss_r50_fpn_1x_street_visualize.py"
cfg = mmcv.Config.fromfile(config_file)
dataset = build_dataset(cfg.data.train)

print("dataset size:", len(dataset))
classnames = ('background',) + (dataset.CLASSES,)

_MEAN = np.array(cfg.img_norm_cfg.mean)
_STD = np.array(cfg.img_norm_cfg.std)
print('dataset keys: ', dataset.__dict__.keys())
print('dataset coco: ', dataset.coco.__dict__.keys())

ln: failed to create symbolic link 'data/datav2': File exists
loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: 'data/annotations/train_night_rain_v2.json'

In [ ]:
# Get a sample
idx = 2
sample = dataset.__getitem__(idx)
print(sample.keys())

img_metas = sample['img_metas'].data
print(img_metas)
print(sample['gt_labels'].data.tolist())
labels = itemgetter(*sample['gt_labels'].data)(dataset.CLASSES)
print(labels)

In [ ]:
# # draw single image
# # Get image
# # visual lize img by name
# name = 'IMG_1360.JPG'
# # for key, value in dataset.coco.imgs.items():
# #     if value['file_name'] == name:
# #         img_id = value['id']
# #         print('value is: ',value)
# #         break
# # print(img_id)
# # sample = dataset.__getitem__(img_id)

# for i, (key, value) in enumerate(dataset.coco.imgs.items()):
#     if value['file_name'] == name:
#         img_id = i
#         break
# print(img_id)
# sample = dataset.__getitem__(img_id)
# Get a sample
idx = 2
sample = dataset.__getitem__(idx)
imgage = sample['img'].data
image = imgage.cpu().numpy().transpose((1,2,0))
image = np.clip(image * _STD + _MEAN, 0, 255).astype('uint8')
print(image.shape)
# image = cv2.rotate(image, cv2.ROTATE_180)
# Draw bboxes
gt_bboxes = sample['gt_bboxes'].data.numpy()
labels_name = itemgetter(*sample['gt_labels'].data)(dataset.CLASSES)
image = cvut.draw_bboxes(image, gt_bboxes, labels=sample['gt_labels'].data, classnames=dataset.CLASSES)

print("Number of bboxes: {}".format(len(gt_bboxes)))

# Plot
plt.figure(figsize=(10,10))
plt.imshow(image)
plt.show()

In [ ]:
# draw whole dataset
for idx in range(len(dataset)):
    sample = dataset.__getitem__(idx)
    # Get image
    img = sample['img'].data
    image = img.cpu().numpy().transpose((1,2,0))
    image = np.clip(image * _STD + _MEAN, 0, 255).astype('uint8')
    
    # Draw bboxes
    gt_bboxes = sample['gt_bboxes'].data.numpy()
#     image = cvut.draw_bboxes(image, gt_bboxes)
    labels_name = itemgetter(*sample['gt_labels'].data)(dataset.CLASSES)
    print(sample['img_metas'].data['filename'])
    image = cvut.draw_bboxes(image, gt_bboxes, labels=sample['gt_labels'].data, classnames=dataset.CLASSES)
    print("Number of bboxes: {}".format(len(gt_bboxes)))

    # # Draw mask
    # gt_semsegs = sample['gt_semsegs'].data.numpy()
    # gt_semsegs = cv2.resize(gt_semsegs[0], image.shape[:2][::-1])
    # image = cvut.draw_masks_overlay(image, [gt_semsegs])

    # Plot
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    plt.show()

In [ ]:
# statistics based on dataset
num_classes = np.zeros(5, dtype=np.int)
for idx in tqdm.tqdm(range(len(dataset))):
    sample = dataset.__getitem__(idx)
    labels = sample['gt_labels'].data
    num_classes[labels] += 1
print(num_classes)

In [ ]:
# statistic based on json
import json
import numpy as np

train_file = '/home/member/Workspace/tank/TS/data/annotations/train.json'
test_file = '/home/member/Workspace/tank/TS/data/annotations/test.json'
all_ = '/home/member/Workspace/tank/TS/data/annotations/thesis_train.json'

with open(train_file) as f:
    train_anno = json.load(f)
with open(test_file) as f:
    test_anno = json.load(f)

with open(all_) as f:
    all_anno = json.load(f)

train_num_elements = np.zeros(5, dtype=np.int)
test_num_elements = np.zeros(5, dtype=np.int)
all_num_elements = np.zeros(5, dtype=np.int)

for an in train_anno['annotations']:
    train_num_elements[an['category_id']-1] += 1
    
for an in test_anno['annotations']:
    test_num_elements[an['category_id']-1] += 1

for an in all_anno['annotations']:
    all_num_elements[an['category_id']-1] += 1

print('train: ', train_num_elements)
print('test : ', test_num_elements)
print('all  : ', all_num_elements)